In [36]:
import configparser
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
import encasm.env
reload(encasm.env)
from encasm.env import PetriDish
import scipy.signal as signal

import utils.videowriter
reload(utils.videowriter)
from utils.videowriter import VideoWriter
import encasm
reload(encasm)
import encasm.eval
reload(encasm.eval)
from encasm.eval import load_tests

data_dir = "./encasm/example_networks/"
test_envs = load_tests(folder = data_dir)

import encasm.env
reload(encasm.env)
from encasm.env import PetriDish

# Loads the config file passed in as a parameter
def load_config(config_file):
    config = configparser.ConfigParser()
    config.read(config_file)
    return config

test_env = PetriDish.from_config_file("test", "./encasm/configs/default_config.ini")

In [37]:
# applies a 2d convolution to the grid using scipy and the given kernel
def convolve(grid, kernel):
    return signal.convolve2d(grid, kernel, mode='same', boundary='fill', fillvalue=0)
    
def diffuse(r,l):
    l = l>0 # convert to boolean in case it isn't

    kernel = np.array([[0,1,0],[1,0,1],[0,1,0]], dtype=np.float32)
    # plt.matshow(r)
    # plt.colorbar()
    # print(r.sum())
    free_cells = (r<0.7) & l # A free cell is alive and isn't full of nutrients
    sources = r > 0.7 # purely cells that are full

    # How large of an area could a cell's resevoir flow to
    flow_area = convolve(free_cells, kernel) * l
    flow_area[l] += 1 # every cell includes itself

    # Nonzero just avoids divide by zero, this is just 5/flow_area * r, which is the adjusted nutrients 
    # to "bounce back" diffusion off of full cells and voids
    to_diff = np.divide(5., flow_area, out=np.zeros_like(r), where=flow_area!=0) * r

    kernel[1,1] = 1
    kernel = kernel * 0.2
    sinks = convolve(to_diff, kernel) * free_cells # sinks serve as sources to other sinks too


    # creates an array where free_cells have the value of sinks and sources have the value of to_diff * 0.2
    diffused = np.zeros_like(r)
    diffused[free_cells] = sinks[free_cells]
    diffused[sources] = to_diff[sources] * 0.2
    # plt.matshow(diffused)
    # plt.colorbar()
    # print(diffused.sum())
    return diffused

In [38]:
# tenv = PetriDish.from_env("tenv", test_envs["small"]['3']['1'])
# tenv.resv += tenv.food

# an 8 by 8 array representing an elbow bend road of life
l = np.array([[0,1,1],
              [0,1,1],
              [0,1,0]], dtype=np.float32)
r = np.array([[0,1,1],
              [0,1,0],
              [0,0,0]], dtype=np.float32)

l = l>0 # convert to boolean in case it isn't

In [46]:
tenv = PetriDish.from_env("tenv", test_envs["large"]['6']['3'])

# Records a video of diffusion in tenv
with VideoWriter(filename="./encasm/results/diffusion.mp4", fps=10) as vid:
    for i in range(1000):
        tenv.resv += tenv.food * 0.2
        tenv.resv = np.clip(tenv.resv, 0, 3)
        tenv.resv = diffuse(tenv.resv, tenv.life)
        # sets 10% of all life cells to 0
        # tenv.resv[tenv.life.nonzero()] *= np.random.rand(*tenv.life[tenv.life.nonzero()].shape) > 0.001
        if i%20 == 0:
            vid.add_concat_grids([tenv.resv, tenv.life], size=750, cmaps = ['hot', 'hot'], cols=2)
            # vid.add_concat_grids([tenv.life,tenv.food,tenv.resv], cmaps=["copper", "hot", "viridis"], cols = 2)
    print(tenv.resv.sum())

6746.241156988327
